# Setup

In [ ]:
import logging
import os
import pandas as pd
import re
import time

from pathlib import Path

%load_ext autoreload
%autoreload 2

# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.FATAL))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("__file__")), '..')))
from setup import *
from tad_help import TadRunner

# Track Layout
Process the infrastructure layout of the Dutch railway system. Requires confidential file. The Layout object provides the routing graph using the block segments of the infrastructure. The regular TrackGraph is also computed.


In [ ]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
if os.path.exists(layout_file):
    start = time.time()
    layout = Layout(layout_file)
    end = time.time()
    print(f"Computed the routing graph for entire Dutch railway network in {end - start} seconds.")
    print(f"Number of track parts in the track graph: {len(layout.g.nodes)} with {len(layout.g.edges)} connections.")
    print(f"Number of blocks in the routing graph: {len(layout.g_block.nodes)} with {len(layout.g_block.edges)} routes.")
    scenario_file_path = Path("../data/prorail/scenarios/TAD/TheHague")
    scenario_files = [f for f in os.listdir(scenario_file_path)]
else:
    print(f"Using dummy file because Dutch railway network file was not found.")
    dummy_file = "../data/dummy_network.json"
    # layout = Layout(dummy_file)
    scenario_file_path = Path("../data/single_track")
    scenario_files = [f for f in os.listdir(scenario_file_path) if "scenario" in f]
save_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", "case_study")
Path(save_dir).mkdir(parents=True, exist_ok=True) 

In [ ]:
os.environ["LOGLEVEL"] = "INFO"

path_data = []
time_data = []
comp_data = []
for scenario_file in scenario_files:
# for scenario_file in ["2025-07-08_3.json"]:
    tad_exp = TadRunner(layout, scenario_file_path / scenario_file, save_dir)
    experiments = tad_exp.run("3500", "o", "GV", "ASDZ", timeout=600)
    experiments[0].metadata = {'color': 'Red',   'label': '@SIPP',    'offset': 0, 'linestyle': 3}
    experiments[1].metadata = {'color': 'Blue',  'label': 'FlexSIPP', 'offset': 0, 'linestyle': 3}
    experiments[2].metadata = {'color': 'Green', 'label': 'rSIPP',    'offset': 0, 'linestyle': 3}

    # tad_exp.plot(experiments, save="3500o-GV")
    # tad_exp.plot([experiments[0]], save="3500o-GV/atsipp")
    # tad_exp.plot([experiments[1]], save="3500o-GV/flexsipp")
    # tad_exp.plot([experiments[2]], save="3500o-GV/rsipp")

    path_data.extend(get_path_data(experiments, tad_exp.agent_df, scenario=scenario_file))
    time_data.extend([exp.get_running_time() | exp.get_label() | {'scenario': scenario_file} for exp in experiments])
    comp_data.extend([exp.get_complexity()   | exp.get_label() | {'scenario': scenario_file} for exp in experiments])

path_df = pd.DataFrame(path_data)
path_df.to_csv(save_dir + "/3500o-Gv.csv")

time_df = pd.DataFrame(time_data)
time_df.to_csv(save_dir + "/3500o-Gv-time.csv")

comp_df = pd.DataFrame(comp_data)
comp_df.to_csv(save_dir + "/3500o-Gv-comp.csv")


In [ ]:
save_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", "tad")
path_df = pd.read_csv(save_dir + "/3500o-Gv.csv", index_col=0)
path_df

In [ ]:
allowed_delay = 180

def td_str(td):
    return ':'.join(re.split(r'[:.]+', str(td)) [1:3])

def extract_tipping_point(df):
    def apply_func(df):

        result=df.groupby("Delay Location").agg({
            "alpha": "min",
            "beta": "max",
            "Delay Amount": "max",
        })
        result = result.loc[result['alpha'] < 900]
        result["Tipping Point"] = result["alpha"].apply(lambda x: td_str(timedelta(seconds=x)))

        def tp_finder(x):
            new_tp = x['beta'] - max(0, x['Delay Amount'] - allowed_delay)
            if new_tp > 0:
                return td_str(timedelta(seconds=new_tp))
            return "-"

        # result[f"Tipping Point ({allowed_delay}s)"] = result.apply(tp_finder, axis=1)
        return result.sort_values("Tipping Point", ascending=True).drop(columns=["alpha", "beta"])

    df["Delay Location"] = df["Delay Location"].str.split("|").apply(lambda x: x[0])
    df = df.groupby(by='Train').apply(apply_func, include_groups=False)
    return df

tp_df = path_df.rename(columns={
    "delay_amount": "Delay Amount",
    "delay_location": "Delay Location",
    "trainNumber": "Train",
    "scenario": "Scenario",
    "label": "Label",
})

tp_df["Scenario"] = tp_df["Scenario"].apply(lambda x: x.split(".")[0])
tp_df["Delay Location"] = tp_df["Delay Location"].apply(lambda x: x.split("-")[1])
tp_df = tp_df.groupby(["Scenario"]).apply(extract_tipping_point, include_groups=False)
tp_df = tp_df.loc[tp_df.index.get_level_values('Scenario').astype(str).str.startswith("2025-07-08_3")]
# tp_df = tp_df.loc[tp_df.index.get_level_values('Train').astype(str).str.startswith("43")]
# tp_df = tp_df.loc[tp_df.index.get_level_values('Delay Location').astype(str).str.startswith("Ledn")]
tp_df.to_latex(save_dir + "/3500o-Gv.tex", escape=True)
tp_df

# Experiment Runtime
Take a route of an agent with many stops, and run from start to every stop as an experiment

## Results


### ATF Plot

### Blocking staircase diagram
Showing the route of the agent with the most stops, its quite long.


In [ ]:
tad_exp = TadRunner(layout, scenario_files / "2025-07-21_1.json", save_dir)
experiments = tad_exp.run("3500", "o", "GV", "ASDZ", timeout=1)

In [ ]:
kwargs = {'min_y': 0*60, 'max_y': (120)*60}
for exp in experiments:
    exp.s.plot(exp.agent.id, exp.block_intervals, exp.buffer_times, exp.recovery_times, True, start="LEDN|", end="SHL|", savefig=f"{save_dir}/2025-07-21_1/blocking_staircase_{exp.metadata['label'].replace(' ', '_')}.png", **kwargs)